# Node embeddings

Graph embeddings are the transformation of property graphs to a vector or a set of vectors. Embedding should capture the graph topology, node-to-node relationships, and other relevant information about graphs, subgraphs, and nodes.

Usage:
- Link prediction
- Node classification
- Node clustering (community detection)
- Visualization

## Node classification

The main steps of the node classification technique are:
1. Load the graph.
2. Sample a number of random nodes (just to make the algorithm quicker).
3. Calculate and/or load the node embeddings.
4. Split the graph into a train and test set.
5. Train the ML classifier.
6. Test the ML classifier.

In [ ]:
import os

if not os.path.exists("./data/graph-large"):
    os.makedirs("./data/graph-large")
    
!wget -qO "./data/graph-large/git_nodes.csv" "https://github.com/memgraph/graph-analytics-course/raw/master/lecture-5/node-classification/git_nodes.csv"
!wget -qO "./data/graph-large/git_edges.csv" "https://github.com/memgraph/graph-analytics-course/raw/master/lecture-5/node-classification/git_edges.csv"

In [1]:
import networkx as nx
import pandas as pd
import random
from tqdm import tqdm
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

# Load the graph, 
# Create a NetworkX graph from files
def load_graph(edges_file_path, nodes_file_path):
    edges = pd.read_csv(edges_file_path, sep=',')
    nodes = pd.read_csv(nodes_file_path, sep=',')
    G = nx.from_pandas_edgelist(edges, 'id_1', 'id_2')
    nx.set_node_attributes(G, pd.Series(nodes.developer_type, 
        index=nodes.id).to_dict(), 'developer_type')
    nx.set_node_attributes(G, pd.Series(nodes.id, 
        index=nodes.id).to_dict(), 'id')
    return G

# Sampling random nodes
# Select defined number of random nodes from a graph
def sample_graph(G, number_of_samples, seed):
    random.seed(seed)
    H = G.copy()
    samples = random.sample(list(G.nodes()), number_of_samples)
    for n in tqdm(G):
        if n not in samples:
            H.remove_node(n)
    H = StellarGraph.from_networkx(H)
    return H

# Load node embeddings from a file into a dictionary
def load_embedding(file_path):
    embedding_dict = {}
    first_line = True
    with open(file_path) as f:
        for line in f:
            if first_line:
                first_line = False
                continue
            vector = [float(i) for i in line.strip().split()]
            embedding_dict[int(vector[0])] = vector[1:]
        f.close()
    return embedding_dict

# Calculate and/or load the node embeddings
def calculate_embeddings(recalculate_embeddings, G, embeddings_file_path):
    if recalculate_embeddings == True:
        rw = BiasedRandomWalk(G)
        
        walks = rw.run(nodes=list(G.nodes()), length=32, n=10, p=0.5, q=2.0)
        print("Number of random walks: {}".format(len(walks)))
        str_walks = [[str(n) for n in walk] for walk in walks]
        
        model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0,
            sg=1, workers=2, epochs=1)
        model.wv.save_word2vec_format(embeddings_file_path)
    return load_embedding(embeddings_file_path)

# Split the graph into a train and test set
def split_data(G_nx, embeddings):
    X = []
    y = []
    for x in embeddings.keys():
        X.append(embeddings[x])
        y.append(G_nx.nodes[x]['developer_type'])
        
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test

# Train the ML classifier
def train_classifier(X_train, y_train):
    clf = LogisticRegressionCV(Cs=10, cv=10, scoring='accuracy',
        verbose=False, max_iter=3000)
    clf.fit(X_train, y_train)
    return clf

# Test the ML classfier
def test_classifier(X_test, y_test, clf):
    y_pred = clf.predict(X_test)
    print(f"Accuracy classification score: {accuracy_score(y_test, y_pred)}")

def main():
    G_nx = load_graph('./data/graph-large/git_edges.csv', 
        './data/graph-large/git_nodes.csv')
    G = sample_graph(G_nx, 10000, 0)
    embeddings = calculate_embeddings(True, G, 
        './data/graph-large/embeddings_node_classify.txt')
    X_train, X_test, y_train, y_test = split_data(G_nx, embeddings)
    clf = train_classifier(X_train, y_train)
    test_classifier(X_test, y_test, clf)
    
if __name__ == "__main__":
    main()

2021-10-09 02:44:26.530885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-09 02:44:26.608292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-09 02:44:26.609333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-09 02:44:26.611051: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Number of random walks: 100000
Accuracy classification score: 0.7915


## Link prediction

The main steps of the link prediction technique are:
1. Load the graph.
2. Sample a number of random nodes (just to make the algorithm quicker).
3. Calculate and/or load the node embeddings.
4. Split the graph into a train and test set.
5. Train the ML classifier.
6. Test the ML classifier.

In [2]:
import pandas as pd
import networkx as nx
import random
from tqdm import tqdm
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from stellargraph.data import EdgeSplitter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

# Load the graph
# Create a NetworkX graph from files
def load_graph(edges_file_path, nodes_file_path):
    edges = pd.read_csv(edges_file_path, sep=',')
    nodes = pd.read_csv(nodes_file_path, sep=',')
    G = nx.from_pandas_edgelist(edges, 'id_1', 'id_2')
    nx.set_node_attributes(G, pd.Series(nodes.developer_type, 
        index=nodes.id).to_dict(), 'developer_type')
    nx.set_node_attributes(G, pd.Series(nodes.id, 
        index=nodes.id).to_dict(), 'id')
    return G

# Sampling random nodes
# Select definied number of random nodes from a graph
def sample_graph(G, number_of_samples, seed):
    random.seed(seed)
    H = G.copy()
    samples = random.sample(list(G.nodes()), number_of_samples)
    for n in tqdm(G):
        if n not in samples:
            H.remove_node(n)
    H = StellarGraph.from_networkx(H)
    return H
    
# Load node embeddings from a file into a dictionary
def load_embedding(file_path):
    embedding_dict = {}
    first_line = True
    with open(file_path) as f:
        for line in f:
            if first_line:
                first_line = False
                continue
            vector = [float(i) for i in line.strip().split()]
            embedding_dict[int(vector[0])] = vector[1:]
        f.close()
    return embedding_dict

# Calculate and/or load the node embeddings
def calculate_embeddings(recalculate_embeddings, G, embeddings_file_path):
    if recalculate_embeddings == True:
        rw = BiasedRandomWalk(G)
        
        walks = rw.run(nodes=list(G.nodes()), length=32, n=10, p=0.5, q=2.0)
        print("Number of random walks: {}".format(len(walks)))
        str_walks = [[str(n) for n in walk] for walk in walks]
        
        model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0,
            sg=1, workers=2, epochs=1)
        model.wv.save_word2vec_format(embeddings_file_path)
    return load_embedding(embeddings_file_path)

# Split the graph into a train and test set
def split_data(G):
    edge_splitter_test = EdgeSplitter(G)
    graph_test, X_test, y_test = edge_splitter_test.train_test_split(
        p=0.1, method='global')
    
    edge_splitter_train = EdgeSplitter(graph_test, G)
    _, X_train, y_train = edge_splitter_train.train_test_split(
        p=0.1, method='global')
    return X_train, y_train, X_test, y_test

def operator_avg(u, v):
    u = np.array(u)
    v = np.array(v)
    return (u + v) / 2.0

# Transform links to features
def link_examples_to_features(X_train, embeddings, binary_operator):
    return [binary_operator(embeddings[src], embeddings[dst])
        for src, dst in X_train
    ]

# Train the ML classifier
def train_classifier(X_train, y_train, embeddings, binary_operator):
    clf = LogisticRegressionCV(Cs=10, cv=10, scoring='roc_auc',
        max_iter=30000)
    X_features = link_examples_to_features(X_train, 
        embeddings, binary_operator)
    clf.fit(X_features, y_train)
    return clf

def evaluate_roc_auc(clf, X_features, y):
    predicted = clf.predict_proba(X_features)
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(y, predicted[:, positive_column])

# Test the ML clssifier
def test_classifier(X_test, y_test, embeddings, binary_operator, clf):
    X_features = link_examples_to_features(X_test, 
        embeddings, binary_operator)
    score = evaluate_roc_auc(clf, X_features, y_test)
    print(f"ROC AUC score: {score}")
    
def main():
    G_nx = load_graph('./data/graph-large/git_edges.csv', 
        './data/graph-large/git_nodes.csv')
    G = sample_graph(G_nx, 10000, 0)
    embeddings = calculate_embeddings(True, G, 
        './data/graph-large/embeddings_link_predict.txt')
    X_train, y_train, X_test, y_test = split_data(G)
    clf = train_classifier(X_train, y_train, embeddings, operator_avg)
    test_classifier(X_test, y_test, embeddings, operator_avg, clf)
    
if __name__ == "__main__":
    main()

100%|██████████| 37700/37700 [00:05<00:00, 6323.51it/s]


Number of random walks: 100000
** Sampled 2154 positive and 2154 negative edges. **
** Sampled 1938 positive and 1938 negative edges. **
ROC AUC score: 0.9168214175177964
